# Chatting bot Intent Detection

<b style = "color:red">Important!</b>

**This demo could only be run with Internet Connetion. Because our dataset and part of the model is online**

**Check the `setup.ipynb` to find the modules needed**

Dataset url https://github.com/google-research-datasets/Taskmaster/raw/master/TM-1-2019

In [1]:
__author__ = ["Haolin Pan", "Riade Benbaki"]
__version__ = "École Polytechnique, 2020/3/31"
__data__ = "https://github.com/google-research-datasets/Taskmaster/raw/master/TM-1-2019/self-dialogs.json"

# Set up

In [2]:
# set up
from collections import defaultdict
import numpy as np
import pandas as pd
import requests
from scipy import stats
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
tf.compat.v1.enable_eager_execution() #Should be commented for training with tf1 only
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import metrics

from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score

Using TensorFlow backend.


## Local Modules

Functions of Topic Clustering are show in the demo `Demo_topic_clustering.ipynb` 

The class `Intent_classification` is to form the data set used for the intent detection

The class `Intent_detection` is to realize the models of detecting the  intents of one sentences

In [3]:
from src.topic_clustering_model import Topic_clustering
from src.sub_intents_detection import Intent_classifiction
from src.sub_intents_detection import Intent_detection

# Initialization

In [4]:
IC = Intent_classifiction()

module use loaded
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


module elmo loaded


# Data

### Raw Data

In [5]:
IC.data[:2]

[{'conversation_id': 'dlg-00055f4e-4a46-48bf-8d99-4e477663eb23',
  'instruction_id': 'restaurant-table-2',
  'utterances': [{'index': 0,
    'speaker': 'USER',
    'text': "Hi, I'm looking to book a table for Korean fod."},
   {'index': 1,
    'speaker': 'ASSISTANT',
    'text': 'Ok, what area are you thinking about?'},
   {'index': 2,
    'speaker': 'USER',
    'text': 'Somewhere in Southern NYC, maybe the East Village?',
    'segments': [{'start_index': 13,
      'end_index': 49,
      'text': 'Southern NYC, maybe the East Village',
      'annotations': [{'name': 'restaurant_reservation.location.restaurant.accept'}]},
     {'start_index': 13,
      'end_index': 25,
      'text': 'Southern NYC',
      'annotations': [{'name': 'restaurant_reservation.location.restaurant.accept'}]}]},
   {'index': 3,
    'speaker': 'ASSISTANT',
    'text': "Ok, great.  There's Thursday Kitchen, it has great reviews.",
    'segments': [{'start_index': 20,
      'end_index': 35,
      'text': 'Thursday Ki

We can see that in the raw data, there is some phrases which is mark by the intents of the user, like `restaurant_reservation.location`

So we want to use this label to establish a deep learning model to detect the intents of the user given a sentence

We store the intents associated with one sentences in the data structure of `dict` and store them in `Intent_Classification.phrs2inetnts` 

In [6]:
IC.intents_clustering()

Here we set the intent **'other'** to the sentence without any intents in the topic.


<div style= "color : red "><b> Attention</b> </div>
Here, the opening phrase of a conversation is marked as **'other'** too. Because they introduce no specific intents other than the topic which we already know

In [7]:
IC.phrs2intents

{"Hi, I'm looking to book a table for Korean fod.": ['other'],
 'Somewhere in Southern NYC, maybe the East Village?': ['restaurant_reservation.location.restaurant.accept'],
 "That's great. So I need a table for tonight at 7 pm for 8 people. We don't want to sit at the bar, but anywhere else is fine.": ['restaurant_reservation.type.seating',
  'restaurant_reservation.time.reservation',
  'restaurant_reservation.num.guests'],
 'What times are available?': ['other'],
 "Yikes, we can't do those times.": ['other'],
 'Let me check.': ['other'],
 'Lets try Boka, are they free for 8 people at 7?': ['restaurant_reservation.name.restaurant.accept',
  'restaurant_reservation.num.guests.accept',
  'restaurant_reservation.time.reservation.accept'],
 "Great, let's book that.": ['other'],
 "No, that's it, just book.": ['other'],
 'Yes please.': ['other',
  'coffee_ordering.preference.accept',
  'coffee_ordering.preference'],
 'Hi I would like to see if the Movie What Men Want is playing here.': ['mov

# Word Embedding

We embed these sentences with the model "elmo", and store the phrases embedded in a `.json` file

In [8]:
IC.word_embedding()

intents clustering completed!
embedded data loading completed
Embedding Progress:  0.0
Embedding Progress:  0.023909716908951797
Embedding Progress:  0.047819433817903594
Embedding Progress:  0.0717291507268554
Embedding Progress:  0.09563886763580719
Embedding Progress:  0.11954858454475899
Embedding Progress:  0.1434583014537108
Embedding Progress:  0.16736801836266257
Embedding Progress:  0.19127773527161437
Embedding Progress:  0.21518745218056617
Embedding Progress:  0.23909716908951797
Embedding Progress:  0.2630068859984698
Embedding Progress:  0.2869166029074216
Embedding Progress:  0.3108263198163734
Embedding Progress:  0.33473603672532515
Embedding Progress:  0.358645753634277
Embedding Progress:  0.38255547054322875
Embedding Progress:  0.4064651874521806
Embedding Progress:  0.43037490436113235
Embedding Progress:  0.4542846212700842
Embedding Progress:  0.47819433817903595
Embedding Progress:  0.5021040550879877
Embedding Progress:  0.5260137719969395
Embedding Progress: 

Finally, we get the data we need to build the model of intent detection, a `p2i` mapping the sentences to its intents and `p2v` mapping the intentces to the its vector representation

In [9]:
p2i, p2v = IC.get_embedded_data_and_clustered_data()

# Modeling

## Initialization the models

In [10]:
ID = Intent_detection(p2i, p2v)

module elmo loaded


## Training

Now we want to train a 0-1 classfication models for **every topic $\times$ every intents in the topic**

The Input is the vector representation of the sentence and the output is bool value whether the topic contains the 

Firstly, we form a well balanced development set:

**50% positive samples**

**25% negative samples with wrong intents**

**25% negative samples without any intents (intent = 'other')**

For example: 

In [11]:
ID.get_data('restaurant', 'time')

We get: 
2257  postive sample
1129  sample of wrong intents
1128  sample of no intents


(array([[ 0.47903794,  0.19902727,  0.09933247, ..., -0.11180887,
          0.04152172, -0.06517152],
        [ 0.07155436, -0.2274629 , -0.08247091, ..., -0.27136704,
          0.2191311 ,  0.09496994],
        [ 0.21138269, -0.4100072 ,  0.2637415 , ..., -0.06332628,
         -0.08134664, -0.09446161],
        ...,
        [ 0.37168708,  0.2541787 , -0.31583005, ..., -0.08804071,
         -0.02512026,  0.22882295],
        [-0.14834785,  0.06300254, -0.06442624, ..., -0.12333031,
          0.21194746,  0.22607255],
        [-0.13581023,  0.04395352, -0.12871231, ..., -0.00336048,
          0.17237607,  0.2781555 ]]), array([0, 1, 1, ..., 0, 0, 0]))

After a simple early_stop research, we find the best early stop epoch within 1 to 10, and then store the best model with the highest `f1_sscore` on the test set

In [58]:
ID.train_model()

Now training the classsfier for topic:  auto  ; intent:  name
Input: str; Output: boolean(if the str contents the intent:  name  ).
----------------------------------------------------------------
We get: 
2616  postive sample
1309  sample of wrong intents
1308  sample of no intents
data_loaded!
Train on 4186 samples
4186/4186 [==============================] - 1s 147us/sample - loss: 0.0483 - mean_absolute_error: 0.2119 - categorical_accuracy: 1.0000
f1_score on dev set: 
f1_macro:  0.9655209004156664
f1_micro:  0.9656160458452722
f1_weighted:  0.9655710680058222
Train on 4186 samples
Epoch 1/2
4186/4186 [==============================] - 1s 150us/sample - loss: 0.0426 - mean_absolute_error: 0.1975 - categorical_accuracy: 1.0000
Epoch 2/2
4186/4186 [==============================] - 0s 46us/sample - loss: 0.0132 - mean_absolute_error: 0.0913 - categorical_accuracy: 1.0000
f1_score on dev set: 
f1_macro:  0.9741848111461271
f1_micro:  0.9742120343839542
f1_weighted:  0.9742080309666266

Epoch 7/9
4186/4186 [==============================] - 0s 40us/sample - loss: 0.0037 - mean_absolute_error: 0.0411 - categorical_accuracy: 1.0000
Epoch 8/9
4186/4186 [==============================] - 0s 40us/sample - loss: 0.0032 - mean_absolute_error: 0.0363 - categorical_accuracy: 1.0000
Epoch 9/9
4186/4186 [==============================] - 0s 40us/sample - loss: 0.0026 - mean_absolute_error: 0.0302 - categorical_accuracy: 1.0000
f1_score on dev set: 
f1_macro:  0.975132640533193
f1_micro:  0.9751671442215855
f1_weighted:  0.9751582971219976

Now training the classsfier for topic:  auto  ; intent:  year
Input: str; Output: boolean(if the str contents the intent:  year  ).
----------------------------------------------------------------
We get: 
964  postive sample
483  sample of wrong intents
482  sample of no intents
data_loaded!
Train on 1543 samples
1543/1543 [==============================] - 1s 372us/sample - loss: 0.0682 - mean_absolute_error: 0.2636 - categorical_accuracy: 1

2588/2588 [==============================] - 0s 42us/sample - loss: 0.0075 - mean_absolute_error: 0.0632 - categorical_accuracy: 1.0000
Epoch 8/8
2588/2588 [==============================] - 0s 41us/sample - loss: 0.0057 - mean_absolute_error: 0.0516 - categorical_accuracy: 1.0000
f1_score on dev set: 
f1_macro:  0.9660442073170732
f1_micro:  0.9660493827160492
f1_weighted:  0.9660429134673291
Train on 2588 samples
Epoch 1/9
2588/2588 [==============================] - 1s 237us/sample - loss: 0.0591 - mean_absolute_error: 0.2638 - categorical_accuracy: 1.0000
Epoch 2/9
2588/2588 [==============================] - 0s 41us/sample - loss: 0.0252 - mean_absolute_error: 0.1470 - categorical_accuracy: 1.0000
Epoch 3/9
2588/2588 [==============================] - 0s 42us/sample - loss: 0.0171 - mean_absolute_error: 0.1151 - categorical_accuracy: 1.0000
Epoch 4/9
2588/2588 [==============================] - 0s 42us/sample - loss: 0.0134 - mean_absolute_error: 0.0955 - categorical_accuracy: 1.0

1975/1975 [==============================] - 0s 42us/sample - loss: 0.0137 - mean_absolute_error: 0.1054 - categorical_accuracy: 1.0000
Epoch 7/7
1975/1975 [==============================] - 0s 42us/sample - loss: 0.0109 - mean_absolute_error: 0.0934 - categorical_accuracy: 1.0000
f1_score on dev set: 
f1_macro:  0.9372158333845806
f1_micro:  0.937246963562753
f1_weighted:  0.9372101733521856
Train on 1975 samples
Epoch 1/8
1975/1975 [==============================] - 1s 307us/sample - loss: 0.0912 - mean_absolute_error: 0.3507 - categorical_accuracy: 1.0000
Epoch 2/8
1975/1975 [==============================] - 0s 42us/sample - loss: 0.0428 - mean_absolute_error: 0.2168 - categorical_accuracy: 1.0000
Epoch 3/8
1975/1975 [==============================] - 0s 42us/sample - loss: 0.0295 - mean_absolute_error: 0.1674 - categorical_accuracy: 1.0000
Epoch 4/8
1975/1975 [==============================] - 0s 42us/sample - loss: 0.0219 - mean_absolute_error: 0.1408 - categorical_accuracy: 1.00

266/266 [==============================] - 0s 67us/sample - loss: 0.0664 - mean_absolute_error: 0.3046 - categorical_accuracy: 1.0000
Epoch 5/6
266/266 [==============================] - 0s 64us/sample - loss: 0.0539 - mean_absolute_error: 0.2597 - categorical_accuracy: 1.0000
Epoch 6/6
266/266 [==============================] - 0s 67us/sample - loss: 0.0450 - mean_absolute_error: 0.2342 - categorical_accuracy: 1.0000
f1_score on dev set: 
f1_macro:  0.7425988700564973
f1_micro:  0.746268656716418
f1_weighted:  0.7448924867189477
Train on 266 samples
Epoch 1/7
266/266 [==============================] - 1s 2ms/sample - loss: 0.2439 - mean_absolute_error: 0.5081 - categorical_accuracy: 1.0000
Epoch 2/7
266/266 [==============================] - 0s 69us/sample - loss: 0.2438 - mean_absolute_error: 0.5075 - categorical_accuracy: 1.0000
Epoch 3/7
266/266 [==============================] - 0s 65us/sample - loss: 0.2438 - mean_absolute_error: 0.5075 - categorical_accuracy: 1.0000
Epoch 4/7
26

...

2614/2614 [==============================] - 1s 239us/sample - loss: 0.0772 - mean_absolute_error: 0.3184 - categorical_accuracy: 1.0000
Epoch 2/3
2614/2614 [==============================] - 0s 41us/sample - loss: 0.0331 - mean_absolute_error: 0.1810 - categorical_accuracy: 1.0000
Epoch 3/3
2614/2614 [==============================] - 0s 39us/sample - loss: 0.0244 - mean_absolute_error: 0.1446 - categorical_accuracy: 1.0000
f1_score on dev set: 
f1_macro:  0.9158688986240109
f1_micro:  0.9159021406727829
f1_weighted:  0.915858670301312
Train on 2614 samples
Epoch 1/4
2614/2614 [==============================] - 1s 235us/sample - loss: 0.0791 - mean_absolute_error: 0.3110 - categorical_accuracy: 1.0000
Epoch 2/4
2614/2614 [==============================] - 0s 41us/sample - loss: 0.0339 - mean_absolute_error: 0.1788 - categorical_accuracy: 1.0000
Epoch 3/4
2614/2614 [==============================] - 0s 41us/sample - loss: 0.0251 - mean_absolute_error: 0.1422 - categorical_accuracy: 1.0

Train on 632 samples
Epoch 1/2
632/632 [==============================] - 1s 877us/sample - loss: 0.1389 - mean_absolute_error: 0.4416 - categorical_accuracy: 1.0000
Epoch 2/2
632/632 [==============================] - 0s 43us/sample - loss: 0.0630 - mean_absolute_error: 0.2639 - categorical_accuracy: 1.0000
f1_score on dev set: 
f1_macro:  0.7969572607055507
f1_micro:  0.8037974683544303
f1_weighted:  0.7964855222470072
Train on 632 samples
Epoch 1/3
632/632 [==============================] - 1s 874us/sample - loss: 0.1257 - mean_absolute_error: 0.4199 - categorical_accuracy: 1.0000
Epoch 2/3
632/632 [==============================] - 0s 41us/sample - loss: 0.0569 - mean_absolute_error: 0.2596 - categorical_accuracy: 1.0000
Epoch 3/3
632/632 [==============================] - 0s 44us/sample - loss: 0.0409 - mean_absolute_error: 0.2263 - categorical_accuracy: 1.0000
f1_score on dev set: 
f1_macro:  0.8531064311411132
f1_micro:  0.8544303797468354
f1_weighted:  0.85292990466035
Train on

data_loaded!
Train on 3374 samples
3374/3374 [==============================] - 1s 196us/sample - loss: 0.0528 - mean_absolute_error: 0.2230 - categorical_accuracy: 1.0000
f1_score on dev set: 
f1_macro:  0.9502369668246445
f1_micro:  0.9502369668246445
f1_weighted:  0.9502369668246445
Train on 3374 samples
Epoch 1/2
3374/3374 [==============================] - 1s 196us/sample - loss: 0.0509 - mean_absolute_error: 0.2224 - categorical_accuracy: 1.0000
Epoch 2/2
3374/3374 [==============================] - 0s 40us/sample - loss: 0.0172 - mean_absolute_error: 0.1141 - categorical_accuracy: 1.0000
f1_score on dev set: 
f1_macro:  0.9549740277972765
f1_micro:  0.9549763033175356
f1_weighted:  0.9549664427297461
Train on 3374 samples
Epoch 1/3
3374/3374 [==============================] - 1s 195us/sample - loss: 0.0316 - mean_absolute_error: 0.1758 - categorical_accuracy: 1.0000
Epoch 2/3
3374/3374 [==============================] - 0s 41us/sample - loss: 0.0135 - mean_absolute_error: 0.0995

548/548 [==============================] - 0s 86us/sample - loss: 0.0228 - mean_absolute_error: 0.1560 - categorical_accuracy: 1.0000
Epoch 6/9
548/548 [==============================] - 0s 86us/sample - loss: 0.0187 - mean_absolute_error: 0.1326 - categorical_accuracy: 1.0000
Epoch 7/9
548/548 [==============================] - 0s 90us/sample - loss: 0.0144 - mean_absolute_error: 0.1138 - categorical_accuracy: 1.0000
Epoch 8/9
548/548 [==============================] - 0s 96us/sample - loss: 0.0117 - mean_absolute_error: 0.0965 - categorical_accuracy: 1.0000
Epoch 9/9
548/548 [==============================] - 0s 62us/sample - loss: 0.0093 - mean_absolute_error: 0.0864 - categorical_accuracy: 1.0000
f1_score on dev set: 
f1_macro:  0.9197801617079744
f1_micro:  0.9202898550724637
f1_weighted:  0.9201508477912393

Now training the classsfier for topic:  uber  ; intent:  duration
Input: str; Output: boolean(if the str contents the intent:  duration  ).
----------------------------------

Epoch 3/9
32/32 [==============================] - 0s 249us/sample - loss: 0.0549 - mean_absolute_error: 0.3056 - categorical_accuracy: 1.0000
Epoch 4/9
32/32 [==============================] - 0s 249us/sample - loss: 0.0334 - mean_absolute_error: 0.2150 - categorical_accuracy: 1.0000
Epoch 5/9
32/32 [==============================] - 0s 312us/sample - loss: 0.0221 - mean_absolute_error: 0.1648 - categorical_accuracy: 1.0000
Epoch 6/9
32/32 [==============================] - 0s 280us/sample - loss: 0.0138 - mean_absolute_error: 0.1225 - categorical_accuracy: 1.0000
Epoch 7/9
32/32 [==============================] - 0s 249us/sample - loss: 0.0088 - mean_absolute_error: 0.0955 - categorical_accuracy: 1.0000
Epoch 8/9
32/32 [==============================] - 0s 280us/sample - loss: 0.0052 - mean_absolute_error: 0.0750 - categorical_accuracy: 1.0000
Epoch 9/9
32/32 [==============================] - 0s 311us/sample - loss: 0.0034 - mean_absolute_error: 0.0617 - categorical_accuracy: 1.0000

### f1 score on test_set

In [59]:
for t in ID.best_f1.keys():
    print(128 * "=")
    print("In topic ", t)
    for st in ID.best_f1[t].keys():
        print("\t", st, ": %.3f"%ID.best_f1[t][st])

In topic  auto
	 name : 0.980
	 year : 0.987
	 reason : 0.966
	 date : 0.950
In topic  coffee
	 size : 0.943
	 location : 0.968
	 num : 0.776
	 name : 0.927
	 type : 0.944
	 preference : 0.869
In topic  movie
	 time : 0.964
	 location : 0.944
	 num : 0.972
	 name : 0.924
	 type : 0.956
	 price : 0.829
In topic  pizza
	 size : 0.953
	 location : 0.940
	 preference : 0.882
	 name : 0.891
	 type : 0.941
In topic  restaurant
	 time : 0.961
	 location : 0.950
	 num : 0.963
	 name : 0.923
	 type : 0.933
In topic  uber
	 time : 0.937
	 location : 0.976
	 num : 0.920
	 duration : 1.000
	 type : 0.931


### Early stop epochs

In [60]:
for t in ID.best_f1.keys():
    print(128 * "=")
    print("In topic ", t)
    for st in ID.best_f1[t].keys():
        print("\t", st, ": %d"%ID.best_epoch[t][st])

In topic  auto
	 name : 6
	 year : 2
	 reason : 7
	 date : 9
In topic  coffee
	 size : 9
	 location : 4
	 num : 8
	 name : 5
	 type : 9
	 preference : 6
In topic  movie
	 time : 9
	 location : 8
	 num : 6
	 name : 9
	 type : 8
	 price : 2
In topic  pizza
	 size : 6
	 location : 9
	 preference : 8
	 name : 7
	 type : 5
In topic  restaurant
	 time : 6
	 location : 5
	 num : 8
	 name : 6
	 type : 7
In topic  uber
	 time : 9
	 location : 6
	 num : 9
	 duration : 4
	 type : 7


# Sentences Demonstration

In [5]:
ID = Intent_detection()
ID.load_model()

test_phrases = np.array([["Get me a restaurant in paris!"], ["This Evening at 8pm"], ["I want this"],
                    ["We have totally 4"], ["My dad, mom, me and my sister"], ["Do you have some recommends"],
                    ["Thank you! Bye"]])

t = 'restaurant'
print(128 * "=")
for phr in test_phrases:
    intent = []
    res = ID.predict( phr[0] , t  )
    sub_topic_array = list( ID.topic2sub_topic[t].keys() )
    print(phr[0])
    for st in sub_topic_array:
        if res[ ID.topic2sub_topic[t][st] - 1] > 0.4:
            intent.append(st)

    if len(intent) > 0:
        print("Detected intents: ", end =" ")
        for i in intent:
            print(i)
    else:
        print("No intent detected")
    print(128*"-")
print(128 * "=")

module elmo loaded
Loading models of topic:  auto
Loaded weights for model auto : name
Loaded weights for model auto : year
Loaded weights for model auto : reason
Loaded weights for model auto : date
Loading models of topic:  coffee
Loaded weights for model coffee : size
Loaded weights for model coffee : location
Loaded weights for model coffee : num
Loaded weights for model coffee : name
Loaded weights for model coffee : type
Loaded weights for model coffee : preference
Loading models of topic:  movie
Loaded weights for model movie : time
Loaded weights for model movie : location
Loaded weights for model movie : num
Loaded weights for model movie : name
Loaded weights for model movie : type
Loaded weights for model movie : price
Loading models of topic:  pizza
Loaded weights for model pizza : size
Loaded weights for model pizza : location
Loaded weights for model pizza : preference
Loaded weights for model pizza : name
Loaded weights for model pizza : type
Loading models of topic:  re

# Conversation Demonstration

In [6]:
TCM = Topic_clustering()

module use loaded
module elmo loaded
module nnlm loaded
Gnews Swivel loaded
Loaded weights for model elmo
Loaded weights for model use


In [7]:
class Demo_Conversation(object):
    def __init__(self, model_cl, model_id):
        self.model_cl = model_cl
        self.model_id = model_id
        self.classes_arr = ['auto', 'coffee', 'movie', 'non-opening', 'pizza','restaurant','uber']
        self.opening_response = {"auto" : "\tIt seems you want to repair your vehicle", \
                            "coffee" : "\tIt seems you want to order some coffee", \
                            "movie" : "\tIt seems you want to book movie tickets", \
                            "pizza" : "\tIt seems you want to order some pizza",\
                            "restaurant": "\tIt seems you want to book a table",\
                            "uber": "\tIt seems you want to take ride"}

    def inConversationDetected(self, embed = "elmo"):
        
        print("Conversation Test")
        print("Still working on it, not completed")
        print("Input \'~\' to stop")
        print(128 * "=")
        print("\tHello, what can I do for you?")
        print(128 * "-")
        userInput = ""
        current_topic = "non-opening";
        pre_topic = "non-opening"
        userInput = input()
        prediction = self.model_cl.prediction(embed, [userInput])[0]
        current_topic = prediction
        
        while current_topic == "non-opening":
            print("\t Sorry I can't understand what you mean")
            print(128 * "-")
            prediction = self.model_cl.prediction(embed, [userInput])[0]
            current_topic = prediction
            userInput = input()
        
        pre_topic = current_topic
        print("\ttopic predicted: ", current_topic)
        print("\tentering the scenario ", current_topic)
        print(self.opening_response[current_topic])
        
        
        while (userInput != "~"):  
            prediction = self.model_cl.prediction(embed, [userInput])[0]
            if prediction == "non-opening" or prediction == current_topic:
                
                pred_intents = self.model_id.predict( userInput , current_topic )
                sub_topic_array = list( self.model_id.topic2sub_topic[t].keys() )
                for st in sub_topic_array:
                    if pred_intents[ self.model_id.topic2sub_topic[t][st] - 1] > 0.4:
                        intent.append(st)

                if len(intent) > 0:
                    print("\tDetected intents: ", end =" ")
                    for i in intent:
                        print(i, end  = " ")
                else:
                    print("\tNo intent detected") 
                userInput = input()
                continue
            
            print(128 * "-")
            current_topic = prediction
            print("\ttopic predicted: ", current_topic)
            print("\tentering the scenario ", current_topic)
            print(self.opening_response[current_topic])
            userInput = input()
        
        print("Goodbye, thank you for your attention")

In [8]:
DC = Demo_Conversation(TCM, ID)

In [9]:
DC.inConversationDetected()


Conversation Test
Still working on it, not completed
Input '~' to stop
	Hello, what can I do for you?
--------------------------------------------------------------------------------------------------------------------------------
I want a ride
	topic predicted:  uber
	entering the scenario  uber
	It seems you want to take ride
	Detected intents:  type i am at fifth avenue
	Detected intents:  type location I need it for 2 people
--------------------------------------------------------------------------------------------------------------------------------
	topic predicted:  pizza
	entering the scenario  pizza
	It seems you want to order some pizza
I am very hungry
	Detected intents:  type location I need it at 6 am
	Detected intents:  type location location 

KeyboardInterrupt: 